In [20]:

# import required module
import os
import pandas as pd
import numpy as np

import country_converter as coco
cc = coco.CountryConverter()

In [21]:
##Add GDP per capita before removing years
GDPpc_Data = (pd.read_excel("../mpd2020.xlsx", sheet_name = "Full data")
                .drop(columns = ["countrycode"])
                .rename(columns = {"gdppc" : "GDPPC", "country" : "Country", "year" : "Year"})
                .replace({'United States' : 'USA'}))
#GDPpc_Data.head()
GDPpc_Data['GDP'] = GDPpc_Data['GDPPC'] * GDPpc_Data['pop']
GDPpc_Data = GDPpc_Data.replace(0, np.nan).dropna()
GDPpc_Data['lnGDP'] = np.log(GDPpc_Data['GDP'])
GDPpc_Data['dlnGDP'] = GDPpc_Data.groupby('Country', group_keys=False)['lnGDP'].diff(1)
#GDPpc_Data["Country"] = cc.convert(names = GDPpc_Data['Country'], to = "name_short")
GDPpc_Data

,Country,Year,GDPPC,pop,GDP,lnGDP,dlnGDP
3,Afghanistan,1950,1156.0000,8150.00000,9.421400e+06,16.058494,NaN
4,Afghanistan,1951,1170.0000,8284.00000,9.692280e+06,16.086840,0.028346
5,Afghanistan,1952,1189.0000,8425.00000,1.001732e+07,16.119827,0.032986
6,Afghanistan,1953,1240.0000,8573.00000,1.063052e+07,16.179240,0.059413
7,Afghanistan,1954,1245.0000,8728.00000,1.086636e+07,16.201182,0.021943
...,...,...,...,...,...,...,...
21677,Zimbabwe,2014,1594.0000,13313.99205,2.122250e+07,16.870573,0.004676
21678,Zimbabwe,2015,1560.0000,13479.13812,2.102746e+07,16.861340,-0.009233
21679,Zimbabwe,2016,1534.0000,13664.79457,2.096179e+07,16.858212,-0.003127
21680,Zimbabwe,2017,1582.3662,13870.26413,2.194784e+07,16.904179,0.045967


In [22]:
GDPpc_Data = GDPpc_Data[['Year', 'Country', 'GDP', 'dlnGDP']].dropna()

In [24]:
TWWI = pd.DataFrame(columns=['Country', 'Year', 'TWWI', 'dpTWWI'])

# assign directory
directory = 'DOTdata'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
#for i in range(1,187):
    #f = f"DOTdata\Exports_to_Count ({i}).xlsx"
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #---------------------------------------------------------------------------------------------
        Coun = ( pd.read_excel(f, skiprows = 3, nrows = 0, usecols = 'B')
                .columns[0] ) #gets country name only
        Coun = cc.convert(names = Coun, to = "name_short")

        temp = ( pd.read_excel(f, skiprows = 6)
                .drop(columns = ["Unnamed: 0"])
                .rename(columns= {'Unnamed: 1': 'Country'})
                .fillna(0) ) #gets data

        #Coun
        temp = temp[temp['Country']!= "Aruba, Kingdom of the Netherlands"]
        temp = temp[temp['Country']!= "Curaçao, Kingdom of the Netherlands"] #countries that don't matter
        temp = temp[temp['Country']!= "Sint Maarten, Kingdom of the Netherlands"]

        temp.Country = cc.convert(names = temp['Country'], to = "name_short")
        temp = temp[temp["Country"] != "not found"]
        temp = temp.melt(id_vars='Country', var_name="Year", value_name="ExpRec")
        temp.Year = temp.Year.astype(int)
        temp.ExpRec = temp.apply(lambda x: float(x['ExpRec'].split(" ")[0].replace(',','')) if type(x['ExpRec']) == str else x['ExpRec'], axis=1) ## 1.2 e issue solving

        CounGDP = GDPpc_Data[GDPpc_Data.Country == Coun]
        temp2 = temp.merge(CounGDP[['Year', 'GDP']], on ='Year', how = 'left').rename(columns = {'GDP': 'DomGDP'})
        temp2['w'] = temp2.apply(lambda x: np.nan if ( np.isnan(x['DomGDP']) or x['DomGDP'] == 0 ) else x['ExpRec'] / x['DomGDP'], axis=1)  ##calculate w per year # * 1e6 ??
        temp2 = temp2.sort_values(by = ["Country", "Year"])

        TW = ( temp2.groupby('Country')[["Year","w"]]
            .rolling(5, min_periods=5, on='Year', closed = "left")
            .mean().reset_index().drop(columns = "level_1")
            .rename(columns = {'w':'TW'}).fillna(0) ) ##calculating TW

        temp3 = temp2.merge(TW, on= ['Year','Country'], how = "outer").merge(GDPpc_Data[["Country", "Year", "GDP", "dlnGDP"]], on = ["Country", "Year"], how = "left")
        temp3['TWi'] = temp3.apply(lambda x: x['TW'] * x['GDP'], axis = 1)
        temp3['dpTWi'] = temp3.apply(lambda x: x['TW'] * x['dlnGDP'], axis = 1)
        #temp3.tail(50)

        temp4 = temp3.groupby('Year')[['TWi', 'dpTWi']].sum().reset_index().rename(columns = {"TWi": 'TWWI', "dpTWi": 'dpTWWI'})
        temp4['Country'] = Coun
        TWWI = pd.concat([TWWI, temp4])
#==============================================================================
temp3

Belgium-Luxembourg not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Netherlands Antilles not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Belgium-Luxembourg not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not found in regex
Belgium-Luxembourg not found in regex
Eastern Germany not found in regex
Netherlands Antilles not found in regex
Serbia and Montenegro not found in regex
Yugoslavia not fou

,Country,Year,ExpRec,DomGDP,w,TW,GDP,dlnGDP,TWi,dpTWi
0,Afghanistan,1961,0.000000,3.189688e+07,0.000000e+00,0.000000e+00,1.314629e+07,0.008635,0.000000e+00,0.000000e+00
1,Afghanistan,1966,0.000000,3.393264e+07,0.000000e+00,0.000000e+00,1.432526e+07,0.009669,0.000000e+00,0.000000e+00
2,Afghanistan,1967,0.000000,3.709477e+07,0.000000e+00,0.000000e+00,1.473403e+07,0.028135,0.000000e+00,0.000000e+00
3,Afghanistan,1968,0.000000,4.142389e+07,0.000000e+00,0.000000e+00,1.525425e+07,0.034699,0.000000e+00,0.000000e+00
4,Afghanistan,1969,0.000000,4.538746e+07,0.000000e+00,0.000000e+00,1.549319e+07,0.015543,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
10691,Zimbabwe,2016,0.000000,5.746962e+08,0.000000e+00,5.638107e-13,2.096179e+07,-0.003127,1.181848e-05,-1.763317e-15
10692,Zimbabwe,2017,0.000074,5.821672e+08,1.271112e-13,5.638107e-13,2.194784e+07,0.045967,1.237443e-05,2.591675e-14
10693,Zimbabwe,2018,0.000000,5.903175e+08,0.000000e+00,3.011078e-14,2.271535e+07,0.034372,6.839770e-07,1.034982e-15
10694,Zimbabwe,2019,0.148030,NaN,NaN,2.542225e-14,NaN,NaN,NaN,NaN


In [25]:
TWWI.to_csv("TWWI.csv", index = False)